#Lista 2
-------------


**Objetivo**: Treinar modelos de classificação utilizando Random Forest e XGBoost.

**Banco de Dados**: Detecção de Doenças no Fígado


[Link](https://drive.google.com/file/d/1jnLwuv4e_ZeCqluMXs3Mar2TGetumtVv/view?usp=drive_link) para o banco


Descrição do banco:
> Este banco de dados contém registros de 416 pacientes diagnosticados com doença no fígado e 167 pacientes sem a doença. Esta informação está na coluna `Selector`.
>
> Há 10 variáveis no banco:
>
> * age: idade
> * Gender: gênero do paciente
> * TB: Bilirrubina total
> * DB: Bilirrubina direta
> * Alkphos: fosfatase alcalina.
> * Sgpt: transaminase glutâmico-pirúvica sérica (TGP)
> * Sgot: transaminase glutâmico-oxalacética sérica (TGO)
> * TP: Proteína total
> * ALB: Albumina
> * A/G Ratio: Relação Albumina:Globulina


**Exercício**:
1. Carregue o banco de dados e analise suas features.

* Transforme a feature Gender em uma variável dummy.

2. Separe o banco de dados:

* Utilize 80% dos dados para treino e 20% para teste.
3. Treine um modelo de Random Forest:

* Utilize os parâmetros n_estimators=150 e max_depth=5.
* Pergunta: Quais são as duas features mais importantes?

4. Treine um modelo de XGBoost:

* Utilize os parâmetros max_depth=5, learning_rate=0.1 e n_estimators=150.
* Pergunta: Quais são as duas features mais importantes?

5. Construa um relatório comparativo dos dois modelos:

* Utilizando a **precisão da classe de pacientes doentes**, qual modelo tem melhor desempenho na detecção de doença no fígado?
* Utilizando o **f1-score da classe de pacientes doentes**, qual modelo tem melhor desempenho na detecção de doença no fígado?


Obs.: Utilize o **mesmo** conjunto de treino e teste para construir e avaliar ambos os modelos.



##1. Carga do banco
--------------------

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import precision_score, f1_score

# Carregar o dataset
df = pd.read_csv('')

# Verificar os valores únicos da coluna de classe
print(df['class_column'].unique())  # Substitua 'class_column' pelo nome correto da coluna alvo

# Ajustar os valores da classe: 1 -> 0 (saudável), 2 -> 1 (doente)
df['class_column'] = df['class_column'].replace({1: 0, 2: 1})

# Separar as variáveis independentes (X) e dependentes (y)
X = df.drop(columns=['class_column'])  # Substitua 'class_column' pelo nome correto da coluna alvo
y = df['class_column']

# Separar os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=150, max_depth=5, random_state=42)
rf_model.fit(X_train, y_train)

# Treinar o modelo XGBoost
xgb_model = XGBClassifier(n_estimators=150, max_depth=5, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

# Fazer previsões
rf_predictions = rf_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)

# Avaliar precisão e F1-score para pacientes doentes (classe 1)
rf_precision = precision_score(y_test, rf_predictions, pos_label=1)
xgb_precision = precision_score(y_test, xgb_predictions, pos_label=1)

rf_f1 = f1_score(y_test, rf_predictions, pos_label=1)
xgb_f1 = f1_score(y_test, xgb_predictions, pos_label=1)

# Exibir resultados
print("Random Forest - Precisão (classe 1):", rf_precision)
print("XGBoost - Precisão (classe 1):", xgb_precision)

print("Random Forest - F1-score (classe 1):", rf_f1)
print("XGBoost - F1-score (classe 1):", xgb_f1)

FileNotFoundError: [Errno 2] No such file or directory: ''

In [17]:

file_path = 'Indian Liver Patient Dataset (ILPD).csv'
data = pd.read_csv(file_path)

##2. Quebra do banco
--------------------

In [18]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['Selector'])
y = data['Selector']



In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((466, 10), (117, 10), (466,), (117,))

##3. Primeiro modelo
--------------------

In [20]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=150, max_depth=5, random_state=42)

X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()
X_train_encoded['Gender'] = X_train_encoded['Gender'].map({'Male': 1, 'Female': 0})
X_test_encoded['Gender'] = X_test_encoded['Gender'].map({'Male': 1, 'Female': 0})

In [21]:

rf_model.fit(X_train_encoded, y_train)
feature_importances = rf_model.feature_importances_
feature_names = X_train_encoded.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})


##3.1 Interpretação de dois coeficientes
--------------------

In [22]:
top_two_features = feature_importance_df.sort_values(by='Importance', ascending=False).head(2)
top_two_features

,Feature,Importance
4,Alkphos,0.170475
6,Sgot,0.141292


##4. Segundo modelo
--------------------

In [47]:
import xgboost as xgb

# Adjust the labels in y_train and y_test to be [0, 1] instead of [1, 2]
y_train_adjusted = y_train - 1
y_test_adjusted = y_test - 1

# Initialize the XGBoost model with the specified parameters
xgb_model = xgb.XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=150, random_state=42, use_label_encoder=False)

# Train the XGBoost model
xgb_model.fit(X_train_encoded, y_train_adjusted)

# Extract feature importances
xgb_importances = xgb_model.feature_importances_

# Create a DataFrame for feature importances
xgb_feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': xgb_importances})

c:\Users\55319\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:16:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


##4.1 Duas features mais importantes
--------------------

In [27]:
# Sort by importance and get the top two features
top_two_xgb_features = xgb_feature_importance_df.sort_values(by='Importance', ascending=False).head(2)

top_two_xgb_features

,Feature,Importance
3,DB,0.231943
2,TB,0.120696


##5. Avaliação do modelo
--------------------

In [31]:
from sklearn.metrics import classification_report


# Obtenha as previsões dos dois modelos
rf_predictions = rf_model.predict(X_test_encoded)
xgb_predictions = xgb_model.predict(X_test_encoded)

In [42]:
from sklearn.metrics import classification_report
import numpy as np
print("Unique values in rf_predictions:", np.unique(rf_predictions))

# Option 1: Filter out instances with unexpected classes
valid_mask = np.isin(rf_predictions, [0, 1])
y_test_adjusted_filtered = y_test_adjusted[valid_mask]
rf_predictions_filtered = rf_predictions[valid_mask]

# Option 2: Map unexpected classes (if class '2' is supposed to be '1')
rf_predictions_mapped = np.where(rf_predictions == 2, 1, rf_predictions)

Unique values in rf_predictions: [1 2]


In [43]:

import numpy as np
from sklearn.metrics import classification_report

print("Unique values in rf_predictions:", np.unique(rf_predictions))

valid_mask = np.isin(rf_predictions, [0, 1])
y_test_adjusted_filtered = y_test_adjusted[valid_mask]
rf_predictions_filtered = rf_predictions[valid_mask]

rf_report = classification_report(y_test_adjusted_filtered, rf_predictions_filtered, target_names=['Non-Liver', 'Liver'])
print(rf_report)

rf_predictions_mapped = np.where(rf_predictions == 2, 1, rf_predictions)
rf_report_mapped = classification_report(y_test_adjusted, rf_predictions_mapped, target_names=['Non-Liver', 'Liver'])
print(rf_report_mapped)

Unique values in rf_predictions: [1 2]
              precision    recall  f1-score   support

   Non-Liver       0.00      0.00      0.00        79
       Liver       0.23      1.00      0.37        23

    accuracy                           0.23       102
   macro avg       0.11      0.50      0.18       102
weighted avg       0.05      0.23      0.08       102

              precision    recall  f1-score   support

   Non-Liver       0.00      0.00      0.00        87
       Liver       0.26      1.00      0.41        30

    accuracy                           0.26       117
   macro avg       0.13      0.50      0.20       117
weighted avg       0.07      0.26      0.10       117



c:\Users\55319\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\55319\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\55319\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

##5.1 Qual modelo escolhido olhando para a precisão de detecção de doença no fígado?
--------------------

In [48]:
from sklearn.metrics import classification_report, precision_score, f1_score

# Previsões do Random Forest
y_pred_rf = rf_model.predict(X_test_encoded)

# Previsões do XGBoost
y_pred_xgb = xgb_model.predict(X_test_encoded)

# Relatório de classificação do Random Forest
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))

# Relatório de classificação do XGBoost
print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb))



Random Forest Classification Report:
              precision    recall  f1-score   support

           1       0.77      0.91      0.84        87
           2       0.47      0.23      0.31        30

    accuracy                           0.74       117
   macro avg       0.62      0.57      0.57       117
weighted avg       0.70      0.74      0.70       117

XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.57      0.18      0.28        87
           2       0.00      0.00      0.00        30

    accuracy                           0.14       117
   macro avg       0.19      0.06      0.09       117
weighted avg       0.42      0.14      0.21       117



c:\Users\55319\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\55319\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\55319\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

##5.2 Qual modelo escolhido olhando para o f1-score do rótulo de doença do fígado?
--------------------